In [2]:
import pandas as pd
import numpy as np

Open Data

In [7]:
dt = pd.read_csv("data/train.csv")
dt

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0
...,...,...,...,...,...,...,...,...,...
637769,2014-07-31,752.0,4.0,7259.0,576.0,1.0,1.0,0,1.0
637770,2014-07-31,753.0,NaN,9938.0,796.0,1.0,1.0,0,1.0
637771,2014-07-31,754.0,4.0,10564.0,1162.0,1.0,1.0,0,NaN
637772,2014-07-31,755.0,4.0,12302.0,1020.0,1.0,1.0,0,1.0


Clean Data

In [ ]:
dt.

Exploratory Analysis

Feature Engineering

Modelling